In [1]:
import pandas as pd
import pandahouse as ph

In [6]:
# Объявляем параметры подключения к clickhouse
connection = dict(database='default',
                  host='http://clickhouse.beslan.pro:8080',
                  user='student',
                  password='dpo_python_2020')

<h2>2.1 Выявить, сколько усердных учеников есть на платформе.</h2><br>
Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.<br>
Горошиной называется меленькое задание.<br>

In [32]:
# запрос 
que = """
SELECT
  count(st_id) as Count_var
from
  (
    SELECT
      st_id,
      MONTH(timest) as mon,
      YEAR(timest) as yer,
      sum(correct) as sum_corr
    FROM
      peas
    WHERE
      mon = 3
      and yer = 2020
    GROUP BY
      st_id,
      mon,
      yer
    order by
      sum_corr DESC
  )
where
  sum_corr >= 20
"""

In [33]:
que_answer21 = ph.read_clickhouse(query=que, connection=connection)
print ('Колличество решивших свыше 20 горошен =', que_answer21['Count_var'][0],'.')

Колличество решивших свыше 20 горошен = 0 .


Кажется на платформе никто не усердный. Увы...

<h2>2.2 Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:</h2><br>
<br>
*ARPU <br>
*ARPAU <br>
*CR в покупку <br>
*СR активного пользователя в покупку <br>
*CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

Есть маленькие притензии к этому заданию.<br>
1. St_id – не является типом int, а скорее имеет тип данных string<br>
2. Такое ощущение что метка ученика участвующего в тесте создавалась когда человек заходил выполнял новую горошину.<br>
*200 значений в studs<br>
*200 final<br>
*200 значений peas<br>
Похожи ли st_id в каждой строке? Как мы видим да.<br>

In [34]:
que = """
SELECT *
FROM studs
"""
que_answer1 = ph.read_clickhouse(query=que, connection=connection)
que_answer1

que = """
SELECT st_id
FROM peas
"""
que_answer2 = ph.read_clickhouse(query=que, connection=connection)

que = """
SELECT st_id
FROM final_project_check
"""

que_answer3 = ph.read_clickhouse(query=que, connection=connection)

sch=0
for i in range(que_answer1.shape[0]):
    if que_answer1.st_id[i] == que_answer2.st_id[i] and que_answer1.st_id[i] == que_answer3.st_id[i]:
        sch=sch+1
        continue
print (sch, que_answer1.shape[0])

200 200


In [35]:
que= """
select
  test_grp,
  sum(default.final_project_check.money) / count(DISTINCT default.studs.st_id) as ARPU,
  sum(default.final_project_check.money) / count(DISTINCT default.peas.st_id) as ARPAU,
  count(DISTINCT default.final_project_check.st_id) / count(DISTINCT default.studs.st_id)*100 as CR1,
  count(DISTINCT default.final_project_check.st_id) / count(DISTINCT default.peas.st_id)*100 as CR2,
  count(math.math_studs) / count(math.math_money)*100 as CR3
from
  default.studs as stud
  left join default.peas as pe on stud.st_id = pe.st_id
  left join default.final_project_check as chek ON stud.st_id = chek.st_id
  left join (
    select
      DISTINCT default.peas.st_id as math_studs,
      default.peas.subject as subject,
      default.final_project_check.st_id as math_money
    from
      default.peas
      left join default.final_project_check on math_studs = math_money
    where
      subject = 'math'
  ) as math on default.peas.st_id = math.math_money
group by
  test_grp
"""

que_answer22 = ph.read_clickhouse(query=que, connection=connection)
que_answer22

,test_grp,ARPU,ARPAU,CR1,CR2,CR3
0,new,3.832586e+06,3.832586e+06,100.0,100.0,100.0
1,old,4.471207e+06,4.471207e+06,100.0,100.0,100.0


где:<br>
CR1 - CR в покупку <br>
CR2 - СR активного пользователя в покупку <br>
CR3 - CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

Извените за мою придирчивость %) <br>
Решил изменить спомощью запросов таблички, для интереса. (уменьшил колличество покупок с пользователя)

In [37]:
que= """
select
  grp,
  sum(money) / count(grp) ARPU,
  sum(money) / sum(if(count_active > 0, 1, 0)) as ARPAU,
  sum(if(count_buy > 0, 1, 0)) / count(grp) * 100 AS CR1,
  sum(if(count_buy > 0, 1, 0)) / sum(if(count_active > 0, 1, 0)) * 100 AS CR2,
  sum(if(buyM > 0, 1, 0)) / sum(if(actM > 0, 1, 0)) * 100 AS CR3
from(
    select
      b.st_id as id,
      a.num_row as grp,
      b.count_buy as count_buy,
      b.s_money as money,
      d.count_active as count_active,
      b.Math as buyM,
      d.Math as actM
    from(
        select
          if(
            num_row <(
              select
                uniqExact(st_id)
              from
                default.studs
            ) / 2,
            'old',
            'new'
          ) as num_row,
          st_id
        from(
            select
              ROW_NUMBER() OVER() as num_row,
              st_id
            from
              (
                select
                  distinct st_id
                from
                  default.studs
              )
          )
      ) as a
      right join (
        select
          st_id,
          groupArray(DISTINCT subject) as subject_active,
          count(subject) as count_buy,
          sum(money) as s_money,
          sum(if (subject = 'Math', 1, 0)) as Math
        from(
            select
              *
            from(
                SELECT
                  ROW_NUMBER() OVER(PARTITION BY st_id) as row_num,
                  *
                from
                  default.final_project_check
              )
            WHERE
              row_num < 11 / 3
          )
        group by
          st_id
      ) as b on a.st_id = b.st_id
      right join (
        SELECT
          *
        from(
            SELECT
              st_id,
              groupArray(DISTINCT subject) as subj_buy,
              sum(if (subject = 'Math', 1, 0)) as Math,
              count(subject) as count_active
            from
              default.peas
            group by
              st_id
          )
      ) as d on a.st_id = d.st_id
  )
group by
  grp
"""

que_answer22n = ph.read_clickhouse(query=que, connection=connection)
que_answer22n

,grp,ARPU,ARPAU,CR1,CR2,CR3
0,new,49687.500000,49687.500000,100.0,100.0,72.727273
1,old,51071.428571,51071.428571,100.0,100.0,50.000000


Ну чтож, поменял я CR3 но уже что то :)